In [ ]:
%matplotlib inline
from IPython.display import HTML
import matplotlib.animation
import matplotlib.pyplot as plt

from arm_simulator.hamilton import System
import autograd.numpy as np

In [ ]:
EARTH_GRAVITY_CONSTANT = 9.81  # m/s^2
l = [2.0, 1.0, 0.5]
mass = 0.42

f = lambda q: np.array([
        l[0] * np.sin(q[0]),
        - l[0] * np.cos(q[0]),
        l[0] * np.sin(q[0]) + l[1] * np.sin(q[0] + q[1]),
        - l[0] * np.cos(q[0]) - l[1] * np.cos(q[0] + q[1]),
        l[0] * np.sin(q[0]) + l[1] * np.sin(q[0] + q[1]) + l[2] * np.sin(q[0] + q[1] + q[2]),
        - l[0] * np.cos(q[0]) - l[1] * np.cos(q[0] + q[1]) - l[2] * np.cos(q[0] + q[1] + q[2]),
    ])
M = mass * np.eye(6)
U = lambda q: mass * EARTH_GRAVITY_CONSTANT * f(q)[1]

pendulum = System(f, M, U)

dt = 0.1
ts = [0]
pendulum.reset(np.asarray([1.0, 0.0, 0.0]), np.asarray([0.0, 0.0, 0.0]))
states = [pendulum.state()]

for i in range(100):
    pendulum.step(dt)
    states.append(pendulum.state())
    ts.append(ts[-1] + dt)

X1 = np.asarray([f(q)[0] for p, q in states])
Y1 = np.asarray([f(q)[1] for p, q in states])
X2 = np.asarray([f(q)[2] for p, q in states])
Y2 = np.asarray([f(q)[3] for p, q in states])
X3 = np.asarray([f(q)[4] for p, q in states])
Y3 = np.asarray([f(q)[5] for p, q in states])

In [ ]:
%%capture

fig, ax = plt.subplots()
ax.axis([-(l[0]+l[1]), (l[0]+l[1]), -((l[0]+l[1]) + 0.5), 0.5])
ax.set_aspect('equal')
ax.set_xlabel('X[m]')
ax.set_ylabel('Y[m]')

link1, = ax.plot([], [], 'b-', lw=2)
link2, = ax.plot([], [], 'b-', lw=2)
link3, = ax.plot([], [], 'b-', lw=2)
c = ax.add_patch(plt.Circle((0, 0), 0.1))

def init():
    link1.set_data([], [])
    link2.set_data([], [])
    link3.set_data([], [])
    c.center = (X3[0], Y3[0])

def animate(i):
    link1.set_data([0, X1[i]], [0, Y1[i]])
    link2.set_data([X1[i], X2[i]], [Y1[i], Y2[i]])
    link3.set_data([X2[i], X3[i]], [Y2[i], Y3[i]])
    c.center = (X3[i], Y3[i])

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(X1), init_func=init)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
HTML(ani.to_html5_video())